In [15]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import mysql.connector

In [6]:
df_train = pd.read_csv('C:/Python/Marketing-Job-Data-Scraping/SQL/step-7-Industry Classification/training-data.csv')
df_train

,Type of Industry,Name
0,Advertising and Marketing,Digital Advertising
1,Advertising and Marketing,Content Marketing
2,Advertising and Marketing,Social Media Marketing
3,Advertising and Marketing,Influencer Marketing
4,Advertising and Marketing,Search Engine Optimization (SEO) Services
...,...,...
1264,Manufacturing,Telecommunication Equipment Manufacturing
1265,Manufacturing,Office Supplies and Stationery Manufacturing
1266,Manufacturing,Solar Panel and Renewable Energy Equipment Man...
1267,Manufacturing,Power Generation and Distribution Equipment Ma...


In [8]:
# Feature extraction - Convert industry name into numerical features using TF-IDF
tfidf_vectorizer = TfidfVectorizer()
X_train = tfidf_vectorizer.fit_transform(df_train['Name'])

# Target labels
y_train = df_train['Type of Industry']

# Train a Support Vector Machine (SVM) classifier
classifier = SVC(kernel='linear')
classifier.fit(X_train, y_train)

SVC(kernel='linear')

In [9]:
# Test: Categorize new industry
new = [
    'Insurance'
]
X_new = tfidf_vectorizer.transform(new)
predicted_categories = classifier.predict(X_new)

for industry, category in zip(new, predicted_categories):
    print(f"Industry: {industry} --> Group: {category}")

Industry: Insurance --> Group: Financial Services


In [10]:
# Apply on our data
df = pd.read_csv('C:/Python/Marketing-Job-Data-Scraping/SQL/step-7-Industry Classification/test_industries.csv')

In [11]:
df

,Industry,pred
0,Advertising Services,Advertising and Marketing
1,International Affairs,Government and Public Services
2,Textile Manufacturing,Manufacturing
3,Marketing Services,Advertising and Marketing
4,Computers and Electronics Manufacturing,Manufacturing
...,...,...
106,E-learning,Education and Training
107,Dairy Product Manufacturing,Manufacturing
108,Sporting Goods Manufacturing,Manufacturing
109,Apparel & Fashion,Others


In [12]:
new_industry = df['Industry']

X1 = tfidf_vectorizer.transform(new_industry)

predicted_categories = classifier.predict(X1)
df['pred'] = predicted_categories
df.to_csv('C:/Python/Marketing-Job-Data-Scraping/SQL/step-7-Industry Classification/test_industries.csv', index=False)

In [13]:
df

,Industry,pred
0,Advertising Services,Advertising and Marketing
1,International Affairs,Government and Public Services
2,Textile Manufacturing,Manufacturing
3,Marketing Services,Advertising and Marketing
4,Computers and Electronics Manufacturing,Manufacturing
...,...,...
106,E-learning,Education and Training
107,Dairy Product Manufacturing,Manufacturing
108,Sporting Goods Manufacturing,Manufacturing
109,Apparel & Fashion,Others


In [16]:
# Connect to MySQL database
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="job_scraping"
)

cursor = mydb.cursor()

for index, row in df.iterrows():
    industry = row["Industry"]
    industry_group = row["pred"]
    
    update_query = "UPDATE industry SET industry_group = %s WHERE industry = %s"
    update_values = (industry_group, industry)
    cursor.execute(update_query, update_values)
    

mydb.commit()
cursor.close()
mydb.close()